# Практика: Архитектура GCN (Graph Convolutional Network)
### Цель:
- Понять основы графов и GCN
- Изучить архитектуру GCN
- Реализовать и обучить GCN графовом датасете (Cora)


In [ ]:
!pip install torch-geometric -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.3 MB/s eta 0:00:00


In [ ]:
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx
from torch_geometric.transforms import RandomLinkSplit
import networkx as nx

from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

## Загрузка и визуализация графовых данных

In [ ]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]
print(data)

Processing...


Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


Done!


In [ ]:
G = to_networkx(data, to_undirected=True)
plt.figure(figsize=(10, 7))
nx.draw(G, node_size=30, with_labels=False)
plt.title('Граф Cora')
plt.show()

## Теория: Как работает GCN?
GCN использует локальную агрегацию информации от соседей:
$$ h^{(l+1)} = \sigma(\tilde{D}^{-1/2}\tilde{A}\tilde{D}^{-1/2} h^{(l)} W^{(l)}) $$
- $\tilde{A} = A + I$ — матрица смежности с самосвязями
- $\tilde{D}$ — соответствующая диагональная матрица степеней
- $W^{(l)}$ — обучаемые параметры слоя
- $\sigma$ — функция активации (обычно ReLU)

## Реализация модели GCN

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # TODO

    def forward(self, x, edge_index):
        # TODO

### Обучение GCN модели

In [ ]:
import copy

def train_node_classification(model, data, optimizer, epochs=200, patience=20):
    best_val_acc = 0.0
    best_model_state = None
    epochs_no_improve = 0

    train_losses = []
    val_accuracies = []

    for epoch in range(epochs):
        # TRAIN
        # TODO

        # VALIDATION
        # TODO

        # EARLY STOPPING
        # TODO

    # LOAD BEST MODEL
    # TODO

    return train_losses, val_accuracies



def evaluate_node_classification(model, data, mask):
    """
    Оценка точности модели на подмножестве узлов, заданном маской.

    Args:
        model: обученная GNN-модель
        data: граф
        mask: булевый тензор маски (train_mask, val_mask, test_mask)

    Returns:
        float: accuracy на выбранной части графа
    """
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        pred = out[mask].argmax(dim=1)
        acc = (pred == data.y[mask]).sum().item() / mask.sum().item()
        return acc

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

# Инициализация модели
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-4)

# Обучение
losses, aucs = train_node_classification(model, data, optimizer, epochs=300)

# Оценка на тесте
test_auc = evaluate_node_classification(model, data, data.test_mask)
print(f"Accuracy на тестовой выборке: {test_auc:.4f}")

### Оценка качества модели

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(aucs)
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Валидационная точность")
plt.grid(True)
plt.show()

# Link prediction

# Задача Link Prediction

Задача **link prediction** — определить, существует ли ребро между двумя узлами графа или может появиться в будущем.

### Формальная постановка

Пусть дан граф:

$$
G = (V, E),
$$

где (V) — множество узлов, а (E) — множество рёбер.

Необходимо обучить модель:

$$
f(u, v) \rightarrow {0, 1},
$$

которая для пары узлов ((u, v)) предсказывает:

* **1**, если между ними есть или может появиться ребро
* **0**, если ребра нет



### Для чего используется link prediction?

* рекомендательные системы (друзья, товары, контакты)
* социальные сети (новые связи)
* биоинформатика (взаимодействия белков/генов)
* научные графы (предсказание отсутствующих ссылок)






In [ ]:
# Загрузим граф Cora
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

### Позитивные и негативные примеры

* **Позитивные примеры:**
  пары узлов ((u, v)), между которыми есть ребро:
  $$(u, v) \in E$$

* **Негативные примеры:**
  пары узлов без ребра:
  $$(u, v) \notin E$$



### Как обучается модель?

1. GNN-энкодер вычисляет эмбеддинги узлов:

   $$
   z_u, ; z_v
   $$

2. Декодер оценивает вероятность существования ребра.
   Часто используется скалярное произведение:

   $$
   \hat{y}_{uv} = \sigma(z_u^\top z_v)
   $$

3. Модель обучается минимизировать бинарную cross-entropy:

   $$
   \mathcal{L} = -[y \log(\hat{y}) + (1-y)\log(1-\hat{y})]
   $$



### Почему важно скрывать часть рёбер?

Если модель увидит тестовое ребро в графе, она может «подсмотреть» наличие связи через соседей.

Поэтому `RandomLinkSplit` создаёт:

* **train-граф:** только обучающие рёбра
* **val-граф:** только train рёбра
* **test-граф:** train + val рёбра




In [ ]:
# Применим RandomLinkSplit
transform = RandomLinkSplit(
    is_undirected=True,              # Граф без направлений
    split_labels=True,               # Метки (0/1) будут созданы
    add_negative_train_samples=True  # Добавить отрицательные примеры в train
)

train_data, val_data, test_data = transform(data)


Метки (pos/neg пары) лежат в отдельных структурах:

* `train_pos_edge_label_index`, `train_neg_edge_label_index`
* `val_pos_edge_label_index`, `val_neg_edge_label_index`
* `test_pos_edge_label_index`, `test_neg_edge_label_index`

In [ ]:
# Посмотреть, сколько примеров
print(train_data.pos_edge_label_index.shape)  # Примеры "ребро есть"
print(train_data.neg_edge_label_index.shape)  # Примеры "ребра нет"


In [ ]:
# Определим GCN Encoder
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels, dropout=0.5):
        super().__init__()
        # TODO

    def forward(self, x, edge_index):
        # TODO

In [ ]:
def evaluate_link_prediction(model, data):
    """
    Оценка link prediction модели (dot-product decoder) по AUC и Average Precision.
    """
    model.eval()
    with torch.no_grad():
        # TODO
        return auc

import copy

def train_link_prediction(model, train_data, val_data, optimizer, epochs=100, patience=10):
    """
    Обучение модели link prediction с:
      - binary cross entropy loss
      - валидацией по AUC
      - ранней остановкой
      - сохранением лучших весов модели

    Args:
        model: GNN энкодер (например, GCNEncoder)
        train_data: Data объект для обучения
        val_data: Data объект для валидации
        optimizer: torch.optim.Adam / SGD
        epochs: максимальное число эпох
        patience: сколько эпох ждать улучшения val_loss или val_auc

    Returns:
        train_losses, val_losses, val_aucs
    """

    train_losses = []
    val_losses = []
    val_aucs = []

    best_val_auc = 0.0
    best_model_state = None
    epochs_no_improve = 0

    for epoch in range(epochs):

        # TRAIN
        # TODO

        # loss
        # TODO

        # VALIDATION
        # TODO

        # EARLY STOPPING
        # TODO

    # LOAD BEST MODEL
    # TODO

    return train_losses, val_losses, val_aucs



In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCNEncoder(dataset.num_node_features, 64, dropout=0.5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
train_data, val_data = train_data.to(device), val_data.to(device)

train_losses, val_losses, val_aucs = train_link_prediction(model, train_data, val_data, optimizer, epochs=200)


In [ ]:
# Оценка на тесте
test_auc, test_ap = evaluate_link_prediction(model, test_data)

print(f"ROC-AUC на тестовой выборке: {test_auc:.4f}")
print(f"Precision на тестовой выборке: {test_ap:.4f}")

In [ ]:
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.xlabel("Epoch")
plt.title("Loss during Link Prediction Training")
plt.legend()
plt.grid(True)
plt.show()
